In [1]:
from pyspark import SparkContext
from numpy.testing import assert_equal

In [2]:
sc = SparkContext('local[*]')

22/02/04 18:09:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/04 18:09:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/04 18:09:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Problem 1a [5 pts]

Create a function `month_reviews` that accepts a globstring to the Netflix prize dataset and return an RDD of tuples where the key is the month the rating was given and the value is the number of ratings given for that month. 

For reference, The Netflix dataset is in CSV format wherein each file contains the user ratings per movie. According to `/mnt/localdata/public/netflixprize/download/README`, the files have the following format:

    TRAINING DATASET FILE DESCRIPTION
    ================================================================================

    The file "training_set.tar" is a tar of a directory containing 17770 files, one
    per movie.  The first line of each file contains the movie id followed by a
    colon.  Each subsequent line in the file corresponds to a rating from a customer
    and its date in the following format:

    CustomerID,Rating,Date

    - MovieIDs range from 1 to 17770 sequentially.
    - CustomerIDs range from 1 to 2649429, with gaps. There are 480189 users.
    - Ratings are on a five star (integral) scale from 1 to 5.
    - Dates have the format YYYY-MM-DD.

In [3]:
path = '/mnt/localdata/public/netflixprize/download/training_set/mv_0002*.txt'


In [4]:
def month_reviews(path):
    import re
    from operator import add
    texts = sc.textFile(path)
    return (texts
                   .map(lambda x: re.findall(r'\d{4}-(\d{1,2})-\d{1,2}', x))
                   .flatMap(lambda x: x)
                   .map(lambda x: (x,1))
                   .reduceByKey(add)
                   .map(lambda x: (int(x[0]), x[1]) if x[0][0] != '0' else (int(x[0][1:]), x[1])))

In [5]:
mr = sorted(
    month_reviews(
        '/mnt/localdata/public/netflixprize/download/training_set/'
        'mv_0002*.txt')
    .collect())
assert_equal(len(mr), 12)
assert_equal(
    mr[:5],
    [(1, 431124), (2, 364658), (3, 396998), (4, 400120), (5, 421313)]
)

## Problem 1b [10 pts]

Create a function `hourly_checkin` that returns a Spark RDD of tuples. The first element of the tuple corresponds to the time of day and the second element corresponds to the number of checkins for that time of day. Use the data in `/mnt/localdata/public/yelp/challenge12/yelp_dataset`, which has the following format:

    {
      // string, 22 character business id, maps to business in business.json
      "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

      "time" : {
          // each key corresponds to day of week and hour of day. example: Mon-0 means Monday, 0000H
          // value corresponds to number of checkins for that day of week and hour of day combination
          "Mon-0": 10,
          "Mon-1": 3,
          "Mon-4": 2,
          ...
      }          
    }

In [6]:
def hourly_checkin():
    path = '/mnt/localdata/public/yelp/challenge12/yelp_dataset/yelp_academic_dataset_checkin.json'
    import json
    import re
    from operator import add
    yelp = sc.textFile(path)

    return (yelp
     .map(json.loads)
     .map(lambda x: x.get('time', ''))
     .map(lambda x: [(k,v) for k,v in x.items()])
     .flatMap(lambda x: x)
     .map(lambda x: (int(*re.findall(r'-(\d{1,2})', x[0])), x[1]))
     .reduceByKey(add))

In [7]:
checkins = sorted(hourly_checkin().collect())

assert_equal(
    checkins[:5],
    [(0, 1399692),
     (1, 1468658),
     (2, 1330682),
     (3, 1018358),
     (4, 706797)]
)

## Problem 1c [10 pts]

Create a function `most_forks` that accepts a globstring from the [Github archive](https://www.gharchive.org) data then returns a list of tuples corresponding to the 10 repositories with the most `ForkEvent`s across the files referred to by the globstring. Each tuple comprises the `repo` `name` and the number of `ForkEvent`s. Sort by decreasing number of `ForkEvent`s then user-repo name alphabetically.

In [8]:
def most_forks(files):
    import json
    from operator import add
    text = sc.textFile(files)
    return (text
     .map(json.loads)
     .filter(lambda x: x.get('type', False) == 'ForkEvent')
     .map(lambda x: (x.get('repo', {}).get('name', ''),1))
     .reduceByKey(add)
     .top(10, key=lambda x: (x[1],x[0])))

In [9]:
# files = '/mnt/localdata/public/gharchive/2020-01-01-*.json.gz'
# import json
# from operator import add
# text = sc.textFile(files)
# (text
#  .map(json.loads)
#  .filter(lambda x: x.get('type', False) == 'ForkEvent')
#  .map(lambda x: (x.get('repo', {}).get('name', ''),1))
#  .reduceByKey(add)
#  .top(10, key=lambda x: (x[1],x[0])))

In [10]:
from operator import itemgetter
mf = most_forks('/mnt/localdata/public/gharchive/2020-01-01-*.json.gz')
assert_equal(
    list(map(itemgetter(1), mf[:5])),
    [83, 82, 54, 54, 45]
)
assert_equal(
    sorted(map(itemgetter(0), mf[:5])),
    ['SmartThingsCommunity/SmartThingsPublic',
     'coolsnowwolf/lede',
     'jackfrued/Python-100-Days',
     'nightscout/cgm-remote-monitor',
     'testerSunshine/12306']
)

In [11]:
'test'

'test'